This notebook only includes stock reporting of the histroical years for power sector by using historical_new_capacity. The model periods can be integrated as well by using CAP variable.

In [2]:
import ixmp

# Load the platform

mp = ixmp.Platform()

In [3]:
import message_ix


# Enter the relevant model and scenario name
""
base_scen = '"oPolicy_2206_macro'"
base_model = 'MESSAGEix-Materials'
scen = message_ix.Scenario(mp, base_model, base_scen, cache=True)

In [4]:
import pandas as pd

# Retreive output material intensities and relevant technology names
"output_cap_ret"
output_cap_ret = scen.par("output_cap_ret")
technologies = output_cap_ret["technology"].unique()
technologies = pd.Series(technologies)
technologies = technologies.to_list()

In [5]:
# Retreive duration_period, historical_new_capacity, technical_lifetime and create a
# historical_total_capacity dataframe

duration_period = scen.par("duration_period")
historical_new_capacity = scen.par(
    "historical_new_capacity", filters={"technology": technologies}
)
technical_lifetime = scen.par(
    "technical_lifetime", filters={"technology": technologies}
)
historical_total_capacity = pd.DataFrame(
    columns=["node_loc", "technology", "year_vtg", "value", "unit", "year_act"]
)

In [6]:
import numpy as np

# Create the historical_total_capacity dataframe based on the lifetime.
# historical_total_capacity = historial_new_capacity * duration_period

for y in historical_new_capacity["year_vtg"].unique():
    for n in historical_new_capacity["node_loc"].unique():
        for t in historical_new_capacity["technology"].unique():
            lifetime = technical_lifetime.loc[
                (technical_lifetime["year_vtg"] == y)
                & (technical_lifetime["node_loc"] == n)
                & (technical_lifetime["technology"] == t),
                "value",
            ].values
            capacity = historical_new_capacity.loc[
                (historical_new_capacity["year_vtg"] == y)
                & (historical_new_capacity["node_loc"] == n)
                & (historical_new_capacity["technology"] == t),
                "value",
            ].values

            if (lifetime.size != 0) & (capacity.size != 0):
                lifetime = lifetime[0]
                lifetime = lifetime.astype(np.int32)

                capacity = capacity[0]
                period = duration_period.loc[
                    (duration_period["year"] == y), "value"
                ].values[0]
                period = period.astype(np.int32)
                val = capacity * period

                until = y + lifetime
                df_temp = pd.DataFrame(
                    {
                        "node_loc": n,
                        "technology": t,
                        "year_vtg": y,
                        "value": val,
                        "unit": "GW",
                        "year_act": list(range(y, until, 5)),
                    }
                )

                historical_total_capacity = pd.concat(
                    [df_temp, historical_total_capacity], ignore_index=True
                )
            else:
                continue

In [7]:
# Filter the dataframe for historical years. For the model years CAP variable should be
# used.

first_year = 2020
historical_total_capacity = historical_total_capacity[
    historical_total_capacity["year_act"] < first_year
]

In [8]:
# Modify the dataframes and merge to calculate the stocks

historical_total_capacity.rename(columns={"value": "capacity"}, inplace=True)
historical_total_capacity = historical_total_capacity.drop(["unit"], axis=1)

output_cap_ret = output_cap_ret.drop(["unit"], axis=1)
output_cap_ret.rename(columns={"value": "material_intensity"}, inplace=True)

merged_df = pd.merge(historical_total_capacity, output_cap_ret, how="inner")

# This way we can consider different material intensities for differnet vintage years.
# E.g. for year_vtg 2015 material intensities change

merged_df["material_stock"] = merged_df["material_intensity"] * merged_df["capacity"]

In [9]:
name = base_model + "_" + base_scen + "_detailed_output1.xlsx"
merged_df.to_excel(name)

In [ ]:
merged_df.drop(["time_dest", "material_intensity"], axis=1, inplace=True)
merged_df_sum = merged_df.groupby(
    ["year_act", "node_loc", "technology", "node_dest", "commodity", "level"]
)["capacity", "material_stock"].sum()
merged_df_sum.reset_index(inplace=True)

In [11]:
name = base_model + "_" + base_scen + "_detailed_output2.xlsx"
merged_df_sum.to_excel(name)

In [12]:
# Sum over the differnet power plants

merged_df_sum.drop(["node_dest", "level", "capacity"], axis=1, inplace=True)

merged_df_sum_material = merged_df_sum.groupby(["year_act", "node_loc", "commodity"])[
    "material_stock"
].sum()
merged_df_sum_material = merged_df_sum_material.to_frame()
merged_df_sum_material.reset_index(inplace=True)

In [13]:
name = base_model + "_" + base_scen + "_detailed_output3.xlsx"
merged_df_sum_material.to_excel(name)

In [14]:
# Prepare data to be dumped out as IAMC excel format

final_output = pd.pivot_table(
    merged_df_sum,
    values="material_stock",
    columns=["year_act"],
    index=["node_loc", "commodity"],
)
final_output.reset_index(inplace=True)
final_output.rename(columns={"node_loc": "Region"}, inplace=True)
final_output = final_output.replace(
    ["aluminum", "cement", "steel"],
    ["Non-Ferrous Metlals|Aluminum", "Non-Metallic Minerals|Cement", "Steel"],
)
final_output = final_output.assign(
    Variable=lambda x: "Material Stock|Power Sector|" + x["commodity"]
)
final_output.drop(["commodity"], axis=1, inplace=True)
final_output["Model"] = base_model
final_output["Scenario"] = base_scen
final_output["Unit"] = "Mt"
year_cols = final_output.select_dtypes([np.number]).columns.to_list()
initial_columns = ["Model", "Scenario", "Region", "Variable", "Unit"]
reorder = initial_columns + year_cols
final_output = final_output[reorder]
output_name = base_model + "_" + base_scen + "_power_sector_material.xlsx"
final_output.to_excel(output_name, index=False)

In [ ]:
mp.close_db()